# Notebook: Convert Model Results to Latex

This notebook is used to load the .json files with the model performance in order to convert them into a latex table for the paper

In [162]:
# Todo: Schauen, ob es die Metriken auch bei anderen Modellen gibt
# Todo: 1.000 <- Punkte einfügen
# Todo: Soll bei nur Real gehen
# Todo: Soll bei allen Tasks gehen
# Todo: bei f1 micro etc 3 nachkommastellen
# Schauen, dass es bei jedem Task "eval_f1_micro", "eval_f1_macro", "accuracy"

## Packages

In [163]:
import json
import numpy as np

## Settings / Constants

In [164]:
RESULTS_PATH_BASE = "../07 train models/results_json/results_"
LLMS = ["Llama70B", "GPT-3"]
# , "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"]
ABSA_TASKS = ["aspect_category", "aspect_category_sentiment", "end_2_end_absa", "target_aspect_sentiment_detection"]
SYNTH_COMBINATIONS = {
    "random": [
        {"real": 500, "synth": 500},
        {"real": 500, "synth": 1000},
        {"real": 500, "synth": 1500}
    ], "fixed": [
        {"real": 25, "synth": 475},
        {"real": 25, "synth": 975},
        {"real": 25, "synth": 1975}
    ]}

In [165]:
LLMS_ENCODED = {"GPT-3": "\\textbf{GPT-3.5-turbo}", "Llama70B": "\\textbf{Llama-2-70B}"}
ENCODE_CONDITION = {"fixed": "\\textbf{LRS\\textsubscript{25}}",
                    "random": "\\textbf{LRS\\textsubscript{500}}"}

N_METRICS = {"aspect_category": 6, "aspect_category_sentiment": 6,
             "end_2_end_absa": 6, "target_aspect_sentiment_detection": 5}

## Code

In [166]:
def round_number(num, decimal_places):
    formatted_num = "{:.{}f}".format(num, decimal_places)
    rounded_num_str = "{:.{}f}".format(float(formatted_num), decimal_places)
    return rounded_num_str

def add_thousand_dots(n_sample):
    if isinstance(n_sample, str):
        if '.' in n_sample:
            integer_part, decimal_part = n_sample.split('.')
            formatted_integer_part = "{:,}".format(int(integer_part))
            result = f"{formatted_integer_part}.{decimal_part}"
        else:
            result = "{:,}".format(int(n_sample))
    elif isinstance(n_sample, np.float64):
        result = "{:,}".format(round(n_sample, 1))
    else:
        result = n_sample
    
    return result

### Report Main Metrics

In [167]:
for absa_task in ABSA_TASKS:
    print(absa_task, ":\n\n", "-------#-----#-----#-------")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"

        with open(json_path, 'r') as json_file:
            results = json.load(json_file)
        if absa_task != "target_aspect_sentiment_detection":
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {add_thousand_dots(round_number(results['eval_f1_micro'], 3))} & {add_thousand_dots(round_number(results['eval_f1_macro'], 3))} & {add_thousand_dots(round_number(results['eval_accuracy'], 3))} \\\\")
        else:
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {add_thousand_dots(round_number(results['eval_f1'], 3))} & {add_thousand_dots(round_number(results['eval_accuracy'], 3))} \\\\")

    print("\\hline")
    for llm_idx, llm in enumerate(LLMS):
        for fs_idx, few_shot_condition in enumerate(["fixed", "random"]):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if fs_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                if absa_task != "target_aspect_sentiment_detection":
                    f1_metrics = f"{add_thousand_dots(round_number(results['eval_f1_micro'], 3))} & {add_thousand_dots(round_number(results['eval_f1_macro'], 3))}"
                else:
                    f1_metrics = f"{add_thousand_dots(round_number(results['eval_f1'], 3))}"
                print(
                    f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {f1_metrics} & {add_thousand_dots(round_number(results['eval_accuracy'], 3))} \\\\")

                # print(llm_idx, fs_idx, freq_idx)
            if freq_idx == 2:
                print(
                    "\\arrayrulecolor{gray}\cline{2-"+str(N_METRICS[absa_task]+1)+"}\\arrayrulecolor{black}")
            else:
                print("\\hline")
        print("\hline")
    print("\n\n")

aspect_category :

 -------#-----#-----#-------
\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 0.869 & 0.851 & 0.780 \\
 &  & 1,000 & 0 & 0.890 & 0.877 & 0.814 \\
 &  & 2,000 & 0 & 0.901 & 0.890 & 0.832 \\
\hline
\multirow{6}{*}{\textbf{Llama-2-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 0.756 & 0.744 & 0.608 \\
 &  & 25 & 975 & 0.770 & 0.750 & 0.632 \\
 &  & 25 & 1,975 & 0.773 & 0.761 & 0.646 \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 0.885 & 0.874 & 0.797 \\
 &  & 500 & 1,000 & 0.875 & 0.864 & 0.781 \\
 &  & 500 & 1,500 & 0.837 & 0.824 & 0.743 \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
\hline
\multirow{6}{*}{\textbf{GPT-3.5-turbo}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 0.810 & 0.809 & 0.693 \\
 &  & 25 & 975 & 0.791 & 0.787 & 0.667 \\
 &  & 25 & 1,975 & 0.743 & 0.726 & 0.597 \\
\arrayrulecolor{gray}\cline{2-7}\arra

### Report Metrics Fine-Grained Metrics

#### ACD

In [168]:
absa_task = "aspect_category"
for n_real_idx, n_real in enumerate([500, 1000, 2000]):
    json_path = RESULTS_PATH_BASE + \
        f"only_real_real{n_real}_synth0_{absa_task}_random.json"
    with open(json_path, 'r') as json_file:
        results = json.load(json_file)

    if n_real_idx == 0:
        condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
    else:
        condition_print = " & "

    class_wise_metrics = ""
    for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
        for metric in ["f1", "accuracy"]:
            class_wise_metrics += f"{add_thousand_dots(round_number(results[f'eval_{metric}_{ac}'], 3))} &"
    print(
        f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {class_wise_metrics[:-1]} \\\\")
print("\\hline")
for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
    for llm_idx, llm in enumerate(LLMS):
        for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
            n_real = freq["real"]
            n_synth = freq["synth"]
            json_path = RESULTS_PATH_BASE + llm + \
                f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
            with open(json_path, 'r') as json_file:
                results = json.load(json_file)
            # print(f"results: {absa_task}, {llm}, {few_shot_condition}, n_real: {n_real}, n_synth: {n_synth}", results)
            if absa_task == "TASD":
                f1_metrics = f"{add_thousand_dots(results['eval_f1_micro'])} & {add_thousand_dots(results['eval_f1_macro'])}"
            else:
                f1_metrics = f"{add_thousand_dots(results['eval_f1_micro'])}"

            if llm_idx == 0 and freq_idx == 0:
                llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
            else:
                llm_print = ""

            if freq_idx == 0:
                condition_print = "\\multirow{3}{*}{" + \
                    ENCODE_CONDITION[few_shot_condition]+"}"
            else:
                condition_print = ""

            class_wise_metrics = ""
            for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
                for metric in ["f1", "accuracy"]:
                    class_wise_metrics += f"{add_thousand_dots(round_number(results[f'eval_{metric}_{ac}'], 3))} &"

            print(
                f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {class_wise_metrics[:-1]} \\\\")
            if freq_idx == 2 and llm_idx == 0:
               print(
                "\\arrayrulecolor{gray}\cline{2-14}\\arrayrulecolor{black}")

    print("\hline")

\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 0.800 &0.905 &0.915 &0.927 &0.897 &0.935 &0.794 &0.941 &0.849 &0.979  \\
 &  & 1,000 & 0 & 0.817 &0.913 &0.930 &0.941 &0.917 &0.947 &0.833 &0.953 &0.889 &0.984  \\
 &  & 2,000 & 0 & 0.830 &0.919 &0.936 &0.946 &0.925 &0.952 &0.857 &0.960 &0.903 &0.986  \\
\hline
\multirow{6}{*}{\textbf{Llama-2-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 0.830 &0.914 &0.915 &0.929 &0.909 &0.942 &0.843 &0.957 &0.873 &0.980  \\
 &  & 500 & 1,000 & 0.818 &0.906 &0.901 &0.919 &0.906 &0.940 &0.839 &0.955 &0.857 &0.977  \\
 &  & 500 & 1,500 & 0.786 &0.890 &0.870 &0.897 &0.867 &0.919 &0.772 &0.938 &0.825 &0.973  \\
\arrayrulecolor{gray}\cline{2-14}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 0.854 &0.926 &0.932 &0.942 &0.924 &0.951 &0.865 &0.963 &0.910 &0.986  \\
 &  & 500 & 1,000 & 0.855 &0.927 &0.933 &0.943 &0.926 &0.952 &0.865 &0.963 &0.905 &0.985  \\
 &  & 500 & 1,5

#### ACSA

In [169]:
absa_task = "aspect_category_sentiment"

idx = 0
for ac_idx, aspect_categories in enumerate([["GENERAL-IMPRESSION", "FOOD"], ["SERVICE", "AMBIENCE"], ["PRICE"]]):
    print("Table for:", aspect_categories, "\n\n")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        condition_string = f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 &"
        metrics_class_wise = ""
        for ac in aspect_categories:
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["eval_f1", "eval_accuracy"]:
                    metrics_class_wise += f" {add_thousand_dots(round_number(results[f'{metric}_{ac}-{polarity}'], 3))} &"

        print(condition_string + metrics_class_wise[:-1] + "\\\\")

    print("\\hline")
    for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
        for llm_idx, llm in enumerate(LLMS):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if llm_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                condition_string = f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} &"
                metrics_class_wise = ""
                for ac in aspect_categories:
                    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                        for metric in ["eval_f1", "eval_accuracy"]:
                            metrics_class_wise += f" {add_thousand_dots(round_number(results[f'{metric}_{ac}-{polarity}'], 3))} &"

                print(condition_string + metrics_class_wise[:-1] + "\\\\")
            
                n_col = 10 if ac_idx == 2 else 16
                if freq_idx == 2:
                   print("\\arrayrulecolor{gray}\cline{2-"+str(n_col)+"}\\arrayrulecolor{black}")
        print("\hline")
    idx += 1

Table for: ['GENERAL-IMPRESSION', 'FOOD'] 


\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 0.786 & 0.947 & 0.000 & 0.992 & 0.756 & 0.949 & 0.893 & 0.949 & 0.604 & 0.977 & 0.779 & 0.932 \\
 &  & 1,000 & 0 & 0.818 & 0.955 & 0.056 & 0.992 & 0.770 & 0.952 & 0.910 & 0.957 & 0.750 & 0.983 & 0.812 & 0.941 \\
 &  & 2,000 & 0 & 0.826 & 0.956 & 0.000 & 0.992 & 0.814 & 0.961 & 0.918 & 0.961 & 0.797 & 0.986 & 0.823 & 0.945 \\
\hline
\multirow{6}{*}{\textbf{Llama-2-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 0.651 & 0.932 & 0.248 & 0.988 & 0.652 & 0.943 & 0.726 & 0.908 & 0.547 & 0.972 & 0.658 & 0.920 \\
 &  & 500 & 1,000 & 0.655 & 0.936 & 0.197 & 0.984 & 0.663 & 0.944 & 0.719 & 0.914 & 0.450 & 0.968 & 0.668 & 0.921 \\
 &  & 500 & 1,500 & 0.749 & 0.938 & 0.306 & 0.990 & 0.752 & 0.949 & 0.798 & 0.914 & 0.507 & 0.970 & 0.759 & 0.929 \\
\arrayrulecolor{gray}\cline{2-16}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 

In [170]:
llllllllllllllllllllll

NameError: name 'llllllllllllllllllllll' is not defined